Import Libraries

In [68]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv
import re 
import time
from copy import deepcopy
import ast
# 최신 LangChain 기준
from langchain_core.documents import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
import gradio as gr
from PyPDF2 import PdfReader
from docx import Document

Parameters

In [11]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
serp_api_key = os.getenv("SERP_API_KEY")
perplexity_api_key = os.getenv("PEPLEXITY_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if serp_api_key:
    print(f"serp_api_key exists and begins {serp_api_key[:8]}")
else:
    print("serp_api_key not set")

if perplexity_api_key:
    print(f"perplexity_api_key exists and begins {perplexity_api_key[:8]}")
else:
    print("perplexity_api_key not set")

# GPT 모델 선언
openai = OpenAI()
MODEL = 'gpt-4o'

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBU
serp_api_key exists and begins c3ee9cec
perplexity_api_key exists and begins pplx-r4f


Functions & Test

In [12]:
def extract_text_from_file(file_path):
    """
    PDF 또는 Word(.docx) 파일에서 텍스트를 추출하고, 줄바꿈 및 띄어쓰기를 정제하는 함수

    Args:
        file_path (str): 파일 경로 (PDF 또는 DOCX)

    Returns:
        str: 정제된 텍스트
    """
    ext = os.path.splitext(file_path)[-1].lower()
    text = ""

    if ext == ".pdf":
        try:
            reader = PdfReader(file_path)
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        except Exception as e:
            print(f"[PDF 읽기 오류] {e}")

    elif ext == ".docx":
        try:
            doc = Document(file_path)
            for para in doc.paragraphs:
                text += para.text.strip() + "\n"
        except Exception as e:
            print(f"[DOCX 읽기 오류] {e}")

    else:
        raise ValueError("지원하지 않는 파일 형식입니다. PDF 또는 DOCX만 가능합니다.")

    # 후처리: 줄바꿈 정제
    # 1. 문장 중간의 줄바꿈(\n)은 띄어쓰기로 치환
    text = re.sub(r"(?<!\n)\n(?!\n)", " ", text)

    # 2. 두 개 이상의 연속 줄바꿈은 문단 구분으로 보고 유지 (하나의 \n으로)
    text = re.sub(r"\n{2,}", "\n", text)

    # 3. 연속 공백 정리
    text = re.sub(r"[ \t]{2,}", " ", text)

    return text.strip()


In [13]:
# test extract_text_from_file, PDF 파일 경로
pdf_path = "과업지시서_20250530.pdf"
rfp_text = extract_text_from_file(pdf_path)
rfp_text[:100]

'1AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계 2025. 05. 15 제조AI연구센터 한국생산기술연구원 담당사양및과업제조AI연구센터윤준석TEL: 032-850-0293'

In [14]:
def get_user_input(
    rfp_text=None,
    style_selected=None,
    keywords_input=None,
    client_name=None,
    proposal_title=None,
    user_direction=None
):
    """
    사용자 입력 기반 제안서 생성용 입력값 정리 함수

    Args:
        rfp_text (str): RFP 원문 텍스트
        style_selected (str): 제안서 스타일 ("격식 있는", "신뢰감 있는" 등)
        keywords_input (str): 강조 키워드 쉼표 구분 (예: "AI, LLM, 효율성")
        client_name (str): 고객사명
        proposal_title (str): 제안서 제목
        user_direction (str): 고객 요청 방향성

    Returns:
        dict: 제안서 생성용 파라미터
    """

    if not rfp_text:
        raise ValueError("⚠️ RFP 텍스트는 필수입니다.")

    style_selected = style_selected or "신뢰감 있는"
    keywords_list = [kw.strip() for kw in (keywords_input or "").split(",") if kw.strip()]
    client_name = client_name or "고객사명 미입력"
    proposal_title = proposal_title or "제안서 제목 미입력"
    user_direction = user_direction or ""

    return {
        "rfp_text": rfp_text,
        "style": style_selected,
        "keywords": keywords_list,
        "client_name": client_name,
        "proposal_title": proposal_title,
        "user_direction": user_direction
    }


In [15]:
# ✅ 예시로 RFP 텍스트와 일부 값들을 입력해 실험
sample_rfp_text = rfp_text

user_inputs = get_user_input(
    rfp_text=sample_rfp_text,
    style_selected="근거가 있고 formal하게",
    keywords_input="타사 비교, 자체적인, 벤치마크",
    client_name="EY 컨설팅",
    proposal_title="AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계",
    user_direction="현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요"
)

# ✅ 출력 확인
for k, v in user_inputs.items():
    print(f"\n🔹 {k}:\n{v if not isinstance(v, str) else v[:50]}")



🔹 rfp_text:
1AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계 2025. 05. 15 제조AI

🔹 style:
근거가 있고 formal하게

🔹 keywords:
['타사 비교', '자체적인', '벤치마크']

🔹 client_name:
EY 컨설팅

🔹 proposal_title:
AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계

🔹 user_direction:
현실적으로 운영 가능하고 모두가 납득할만한 근거를 가진 맞춘 설계 필요


In [47]:
PROPOSAL_SLIDE_TEMPLATES = {
    "cover_page": {
        "elements": {
            "Title": "프로젝트의 정식 명칭 (중앙 상단에 크게 배치)",
            "Subtitle": "고객사명 또는 부제 설명 (Title 아래 위치)",
            "ProjectDate": "제안서 작성 또는 제출 일자 (하단 우측 또는 좌측 구석에 배치)",
            "PreparedBy": "작성자 또는 제안 주체 (ProjectDate 인근 또는 하단 중앙)",
            "Logo": "회사 또는 고객사 로고 (우상단 또는 좌상단에 적절히 배치)"
        },
    },
    "table_of_contents": {
        "slide_description": "전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.",
        "description": "",
        "elements": {
            "Title": "목차 제목 (상단 중앙)",
            "SectionList": "슬라이드별 주요 제목 리스트 (Bullet 형식으로 왼쪽 정렬)"
        },
    },
    "executive_summary": {
        "slide_description": "제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (예: 제안 개요, Executive Summary)",
            "MiddleText": "",
            "SummaryPoints": "핵심 제안 내용 Bullet (전략, 기대 효과, 기간, 투자 규모 등)",
            "ClientValue": "고객에게 제공되는 핵심 가치 또는 차별화된 이점 요약"
        },
        "needs_research": []
    },
    "project_understanding": {
        "slide_description": "프로젝트의 필요성과 배경을 설명하며, 고객의 상황과 과제를 명확히 인식하고 있다는 점을 전달합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙)",
            "MiddleText": "프로젝트 요약 또는 핵심 문장 (제목 아래 강조 박스)",
            "KeyObjectives": "고객의 주요 목표 목록 (왼쪽 열에 Bullet 형식)",
            "BackgroundIssues": "해결하고자 하는 문제 또는 현재 상황 (오른쪽 열에 Bullet 형식)"
        },
        "needs_research": []
    },
    "client_needs_summary": {
        "slide_description": "고객의 구체적인 요구사항을 명확하게 정리해 실무적 방향성과 대응의 기준을 제시합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목 (상단 중앙)",
            "MiddleText": "요구사항 요약 핵심 문장",
            "BulletPoints": "고객 니즈 요약 문장 리스트",
            "NeedsMatrix": "요구사항을 정리한 표 (요구사항 | 중요도 | 현재 상태)"
        },
        "needs_research": []
    },
    "market_analysis_market_overview": {
        "slide_description": "해당 산업의 크기, 구조, 주요 트렌드를 데이터와 함께 제시해 전체 시장 배경을 설명합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "시장 개요 요약 문장",
            "GraphLeft": "시장 관련 그래프 (예: 점유율, 성장률 등)",
            "TextRight": "시장 현황 및 주요 특징 설명 텍스트"
        },
        "needs_research": []
    },
    "growth_trend_analysis": {
        "slide_description": "시장 혹은 기술의 성장세를 데이터 기반으로 시각화하고 그 시사점을 제시합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "성장 트렌드 핵심 요약 문장",
            "GraphLeft": "성장 추이 그래프 (선형, 누적 등)",
            "TextRight": "그래프 해석 및 성장 요인 서술"
        },
        "needs_research": []
    },
    "industry_drivers_challenges": {
        "slide_description": "산업 내 성장 요인과 저해 요인을 대조해 전략 방향 설정의 기초로 삼습니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "산업 변화 요약 또는 주요 시사점",
            "DriversList": "산업 동인 목록 (왼쪽 컬럼 Bullet 형식)",
            "ChallengesList": "산업 장애 요소 목록 (오른쪽 컬럼 Bullet 형식)",
            "DetailComments": "각 항목에 대한 보충 설명 또는 전략적 인사이트"
        },
        "needs_research": []
    },
    "competitive_benchmarking": {
        "slide_description": "경쟁사 대비 자사의 상대적 위치와 우위를 정량적·정성적으로 설명합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "경쟁 포지셔닝 요약",
            "TableMain": "경쟁사 비교 요약 테이블",
            "BenchmarkTable": "세부 비교 테이블 (경쟁사 별 행, 항목 별 열 구성)",
            "CompetitiveInsights": "경쟁사 전략 및 고객 시사점 도출",
            "TextBottom": "요약 또는 인사이트 도출 문장"
        },
        "needs_research": []
    },
    "swot_analysis": {
        "slide_description": "내부·외부 환경 분석을 통해 전략적 포지션을 진단하고 방향성을 제시합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "SWOT 분석 요약 문장",
            "SWOT-S": "Strength 요소 리스트",
            "SWOT-W": "Weakness 요소 리스트",
            "SWOT-O": "Opportunity 요소 리스트",
            "SWOT-T": "Threat 요소 리스트",
            "NarrativeSummary": "요약 설명 또는 전략적 시사점"
        },
        "needs_research": []
    },
    "technical_specifications": {
        "slide_description": "제안 솔루션의 기술적 구조, 사용 기술, 플랫폼 등을 상세히 설명합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "",
            "TechStackDiagram": "기술 스택 구조도 (Front-End, Back-End, DB 등)",
            "SystemCompatibility": "플랫폼/브라우저 호환성 설명",
            "SecurityFeatures": "보안 및 인증 체계 요약",
            "PerformanceMetrics": "성능 지표 또는 SLA 항목"
        },
        "needs_research": []
    },
    "service_operation_model": {
        "slide_description": "서비스 운영 및 유지보수 체계를 설명하여 안정성 및 지속 가능성을 강조합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "",
            "OperationOrgChart": "운영 조직도 또는 지원 체계도",
            "SLACommitment": "서비스 수준 협약 (SLA) 요약",
            "MaintenancePlan": "유지보수 및 정기점검 계획",
            "SupportChannels": "지원 채널 및 운영 시간 정보"
        },
        "needs_research": []
    },
    "compliance_and_governance": {
        "slide_description": "법적/정책적 준수 사항 및 거버넌스 체계를 설명합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "",
            "ComplianceList": "준수 기준 리스트 (ISO, ISMS 등)",
            "PolicyDiagram": "보안/정책 흐름도",
            "DataProtectionMeasures": "데이터 보호 및 접근제어 조치",
            "AuditAndMonitoring": "감사 및 모니터링 체계 요약"
        },
        "needs_research": []
    },
    "client_case_references": {
        "slide_description": "과거 수행한 유사 프로젝트 사례를 통해 신뢰성과 경험을 강조합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "",
            "CaseList": "유사 프로젝트 리스트 (고객사, 수행 내용, 성과 등)",
            "VisualHighlights": "대표 사례 이미지 또는 그래프",
            "SuccessIndicators": "주요 성과 지표 요약"
        },
        "needs_research": []
    },
    "solution_overview": {
        "slide_description": "제안 솔루션의 핵심 가치와 구조를 시각화하고 간명하게 전달합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "솔루션 개요 및 핵심 가치 요약 (비즈니스적 표현)",
            "SimpleDiagram": "End-to-End 프로세스 또는 Value Chain 시각화",
            "ArchitectureDiagram": "High-level 기술 아키텍처 (예: 모듈 기반 구성)",
            "KeyModules": "핵심 모듈 및 기능 설명 (각 블록별 1문장)"
        },
        "needs_research": []
    },
    "use_case_scenarios": {
        "slide_description": "솔루션이 실제로 어떻게 작동하고 활용될 수 있는지를 시나리오 기반으로 보여줍니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "",
            "ScenarioDiagram": "워크플로우 또는 사용자 여정 흐름도",
            "NarrativeCases": "상황 기반 시나리오 설명"
        },
        "needs_research": []
    },
    "strategic_recommendations": {
        "slide_description": "고객을 위한 전략적 방향성과 구체 실행안, 우선순위를 제시합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "전략 방향 요약",
            "BulletPoints": "주요 전략 항목",
            "PriorityMap": "2x2 우선순위 매트릭스"
        },
        "needs_research": []
    },
    "implementation_plan": {
        "slide_description": "전략 실행을 위한 단계별 로드맵을 구체적으로 설명합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "실행 로드맵 핵심 요약",
            "TimelineMain": "전체 실행 일정 요약",
            "Phases": "단계별 세부 계획",
            "TextBottom": "리스크 또는 보완 고려사항",
            "ImplementationTeam": "각 단계별 투입 인력 및 담당 역할 (선택사항)"
        },
        "needs_research": []
    },
    "timeline_milestones": {
        "slide_description": "전체 일정에서의 주요 마일스톤을 시각화하여 이해도를 높입니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "일정 요약 또는 주요 포인트",
            "GanttChart": "간트 차트 또는 일정 도식"
        },
        "needs_research": []
    },
    "risk_management_plan": {
        "slide_description": "예상되는 리스크를 식별하고 이에 대한 대응 전략을 제시합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "리스크 관리 개요",
            "RiskTable": "리스크 목록 (내용 | 가능성 | 영향도)",
            "MitigationStrategy": "완화 전략 항목"
        },
        "needs_research": []
    },
    "expected_benefits": {
        "slide_description": "제안서 실행 시 고객이 얻게 될 주요 기대 효과를 정성·정량적으로 표현합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "핵심 기대 효과 요약",
            "BulletPoints": "기대 효과 리스트",
            "KPIProjection": "성과 지표 추이 그래프 또는 테이블 (예: 생산성 향상률, 비용 절감율)"
        },
        "needs_research": []
    },
    "investment_budget_estimation": {
        "slide_description": "전체 예산과 각 항목별 비용을 상세히 제시하여 투자 가시성을 제공합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "투자 요약",
            "BudgetTable": "총 예산 테이블",
            "CostBreakdown": "비용 항목별 상세 내용"
        },
        "needs_research": []
    },
    "team_introduction": {
        "slide_description": "수행팀의 전문성과 역할 분담을 보여줘 신뢰도를 높입니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "팀 구성 및 핵심 인력 요약",
            "MemberList": "핵심 인력 소개 (이름, 역할, 경력 등)",
            "TeamOrgChart": "팀 조직도"
        },
        "needs_research": []
    },
    "why_us_differentiation": {
        "slide_description": "자사만의 차별성과 강점을 강조하여 경쟁사 대비 우위를 설득합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "자사 강점 요약",
            "BulletPoints": "차별화 요소 리스트",
            "ComparisonTable": "자사 vs 경쟁사 비교 표"
        },
        "needs_research": []
    },
    "closing_summary": {
        "slide_description": "전체 제안을 요약하며 고객에게 남기고 싶은 핵심 메시지를 강조합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "MiddleText": "핵심 요약 문장",
            "FinalCall": "콜투액션 또는 향후 협업 제안"
        },
        "needs_research": []
    },
    "qna": {
        "slide_description": "질의응답을 위한 충분한 공간과 예상 질문 대응 내용을 구성합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "EmptySpace": "Q&A 시각적 공간",
            "AnticipatedQuestions": "예상 질문 및 답변 리스트"
        },
        "needs_research": []
    },
    "appendix": {
        "slide_description": "통계, 기술 스펙, 참고 문헌 등 본문에 넣기 어려운 자료를 정리합니다.",
        "description": "",
        "elements": {
            "Title": "슬라이드 제목",
            "SupportingDetails": "기술 자료, 통계 인용 등",
            "Footnotes": "출처, 링크, 각주 등 보충 정보"
        },
        "needs_research": []
    }
}


In [129]:
PROPOSAL_SLIDE_TEMPLATES = {
    "cover_page": {
        "elements": {

        },
    },
    "table_of_contents": {
        "slide_description": "전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.",
        "description": "",
        "elements": {
        },
    },
    "executive_summary": {
        "slide_description": "제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "project_understanding": {
        "slide_description": "프로젝트의 필요성과 배경을 설명하며, 고객의 상황과 과제를 명확히 인식하고 있다는 점을 전달합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "client_needs_summary": {
        "slide_description": "고객의 구체적인 요구사항을 명확하게 정리해 실무적 방향성과 대응의 기준을 제시합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "market_analysis_market_overview": {
        "slide_description": "해당 산업의 크기, 구조, 주요 트렌드를 데이터와 함께 제시해 전체 시장 배경을 설명합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "growth_trend_analysis": {
        "slide_description": "시장 혹은 기술의 성장세를 데이터 기반으로 시각화하고 그 시사점을 제시합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "industry_drivers_challenges": {
        "slide_description": "산업 내 성장 요인과 저해 요인을 대조해 전략 방향 설정의 기초로 삼습니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "competitive_benchmarking": {
        "slide_description": "경쟁사 대비 자사의 상대적 위치와 우위를 정량적·정성적으로 설명합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "swot_analysis": {
        "slide_description": "내부·외부 환경 분석을 통해 전략적 포지션을 진단하고 방향성을 제시합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "technical_specifications": {
        "slide_description": "제안 솔루션의 기술적 구조, 사용 기술, 플랫폼 등을 상세히 설명합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "service_operation_model": {
        "slide_description": "서비스 운영 및 유지보수 체계를 설명하여 안정성 및 지속 가능성을 강조합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "compliance_and_governance": {
        "slide_description": "법적/정책적 준수 사항 및 거버넌스 체계를 설명합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "client_case_references": {
        "slide_description": "과거 수행한 유사 프로젝트 사례를 통해 신뢰성과 경험을 강조합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "solution_overview": {
        "slide_description": "제안 솔루션의 핵심 가치와 구조를 시각화하고 간명하게 전달합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "use_case_scenarios": {
        "slide_description": "솔루션이 실제로 어떻게 작동하고 활용될 수 있는지를 시나리오 기반으로 보여줍니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "strategic_recommendations": {
        "slide_description": "고객을 위한 전략적 방향성과 구체 실행안, 우선순위를 제시합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "implementation_plan": {
        "slide_description": "전략 실행을 위한 단계별 로드맵을 구체적으로 설명합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "timeline_milestones": {
        "slide_description": "전체 일정에서의 주요 마일스톤을 시각화하여 이해도를 높입니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "risk_management_plan": {
        "slide_description": "예상되는 리스크를 식별하고 이에 대한 대응 전략을 제시합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "expected_benefits": {
        "slide_description": "제안서 실행 시 고객이 얻게 될 주요 기대 효과를 정성·정량적으로 표현합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "investment_budget_estimation": {
        "slide_description": "전체 예산과 각 항목별 비용을 상세히 제시하여 투자 가시성을 제공합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "team_introduction": {
        "slide_description": "수행팀의 전문성과 역할 분담을 보여줘 신뢰도를 높입니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "why_us_differentiation": {
        "slide_description": "자사만의 차별성과 강점을 강조하여 경쟁사 대비 우위를 설득합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "closing_summary": {
        "slide_description": "전체 제안을 요약하며 고객에게 남기고 싶은 핵심 메시지를 강조합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "qna": {
        "slide_description": "질의응답을 위한 충분한 공간과 예상 질문 대응 내용을 구성합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    },
    "appendix": {
        "slide_description": "통계, 기술 스펙, 참고 문헌 등 본문에 넣기 어려운 자료를 정리합니다.",
        "description": "",
        "elements": {
        },
        "needs_research": []
    }
}


In [130]:
def analyze_rfp(
    user_inputs: dict,
    slide_templates: dict,
    model="gpt-4o",
    temperature=0.3
):
    """
    get_user_input 결과 dict를 받아 GPT로 분석하여 PPT 슬라이드 구조 생성

    Args:
        user_inputs (dict): get_user_input 함수에서 반환된 사용자 입력값
        slide_templates (dict): PPT 템플릿 구조
        model (str): 사용할 OpenAI 모델
        temperature (float): 생성 온도

    Returns:
        dict: 슬라이드별 제안서 구성 내용
    """
    import openai
    import json
    import re

    rfp_text = user_inputs.get("rfp_text")
    style = user_inputs.get("style", "신뢰감 있는")
    keywords = user_inputs.get("keywords", [])
    client_name = user_inputs.get("client_name", "고객사명 미입력")
    proposal_title = user_inputs.get("proposal_title", "제안서 제목 미입력")
    user_direction = user_inputs.get("user_direction", "")

    if not rfp_text or len(rfp_text.strip()) < 30:
        raise ValueError("❗ RFP 원문이 비어 있거나 너무 짧습니다. 실제 RFP를 반드시 입력하세요.")
    if slide_templates is None:
        raise ValueError("PPT 템플릿 구조 딕셔너리가 필요합니다.")

    # system prompt
    system_prompt = f"""
    당신은 'EY·맥킨지 등 국내외 최상위 전략 컨설팅 회사의 파트너급 제안서 전문가 AI'입니다.
    
    아래 RFP 원문과 고객 요구를 바탕으로,
    각 PPT 슬라이드별로 실제 컨설팅 현장과 동일한 수준의, 논리적이고 설득력 있는 제안서 초안 구조를 작성하세요.
    
    - 슬라이드 순서와 템플릿 활용은 RFP의 논리적 흐름, 고객의 의사결정 포인트, 설득 전략에 따라 자유롭게 조정하세요.
    - 하나의 템플릿을 여러 번 사용하거나, 필요에 따라 생략·병합해도 무방합니다.
    - 각 슬라이드의 elements에는 실제 발표자료처럼 활용 가능한 표, 그래프, 수치, 근거, 사례, 시각적 다이어그램, 실제 데이터, 실무 워딩을 최대한 풍부하게 포함하세요.
    - 모든 슬라이드는 고객사 임원·실무자 모두를 설득할 수 있도록 전략적 논리, 정량/정성 근거, 차별화 요인, 업계 사례, 실무 설득력 기반으로 작성되어야 하며, 평가자와 실무자가 모두 납득할 수 있어야 합니다.
    
    1. [description]
    - 단순 목적 설명이 아닌: '왜 중요한가', '고객사 상황과의 연계성', '전략적 필요성', '차별화된 근거', '시장 및 경쟁사 변화와 연결성'을 반드시 포함하세요.
    - 시장 통계, 업계 사례, 정책·기술 트렌드, 고객사 현황 등 객관적 근거를 적극 인용하세요.
    - 3~5문장 이상의 깊이 있는 발표용 컨설팅 워딩으로 작성하세요.
    - 슬라이드의 논리적 흐름과 고객 의사결정 포인트와의 연결을 명확히 하세요.
    
    2. [elements]
    - 실제 발표자료처럼 사용 가능한 수준으로 표, 수치, 근거, 사례, 시각적 다이어그램, 실제 데이터, 실무 워딩을 최대한 풍부하게 포함하세요.
    - 표, 그래프, 다이어그램 등 시각적 요소는 반드시 포함하고, 실제 데이터와 출처를 명확히 기재하세요.
    - 표는 다음 형식으로 작성:
      "표 제목": [["헤더1", "헤더2"], ["값1", "값2"]]
    - 그래프는 다음 형식으로 작성:
      "그래프 제목": {{
        "description": "시장 성장률 추이 (2019~2024)",
        "graph_type": "Line Chart", (Pie Chart 등)
        "data_source": "Statista, 2023",
        "data_table": [["연도", "시장 규모 (억원)"], ["2019", "1200"], ["2020", "1400"]]
      }}
    - 실제 업계 사례, 벤치마크, 성공/실패 요인, 고객사 맞춤형 인사이트 등도 반드시 포함하세요.
    - description의 핵심 논리와 근거가 elements에도 반드시 반영되어야 하며, 슬라이드의 설득 포인트가 명확히 드러나야 합니다.
    
    3. [needs_research]
    - 외부 리서치가 필요한 항목(정책, 통계, 사례 등)을 질문 형식으로 구체화하세요.
    - 단순 요약이 아니라, description의 논리와 설득을 뒷받침할 수 있는 실질적 조사 질문이어야 합니다.
    - 실제 현업 컨설턴트가 리서치 요청을 할 때 사용하는 수준의 구체적인 질문으로 작성하세요.
    - 질의하는 주체가 무엇인지 명확히 하세요. ex). 기업A -> 애플, 경쟁사: 제안서 상의 관련있는 기업
    - elements내 나온 정보가 명확한지 질문을 생성해주세요.
    
    4. [Title]
    - 각 슬라이드 description과 slide_id에 적합한 내용으로, 한 번에 해당 슬라이드의 핵심 메시지와 의미가 명확히 드러나도록 작성하세요.
    - slide_id와 동일한 표현은 사용하지 마세요. (예: Project Understanding 금지)
    - 고객사 임원·실무자가 슬라이드 제목만 보고도 내용을 직관적으로 이해할 수 있어야 합니다.

    5. [MiddleText]
    - 대부분의 ppt에 들어가는 elements로 Title 아래 최대한 상세히 Title을 설명하는 역할을 합니다.
    
    추가 지침:
    - 슬라이드별로 논리적 연결성(Why→What→How→So What→Next)을 고려해 작성하세요.
    - 고객사 맞춤형 메시지, 업계 트렌드, 경쟁사 동향, 차별화 전략, 실제 수치와 사례, 시각적 설득력을 모두 반영하세요.
    - 슬라이드별로 실제 컨설팅사 파트너가 직접 리뷰·수정하는 수준의 품질을 목표로 하세요.
    - 모든 슬라이드는 실제 데이터, 실제 사례, 실무적 설득력을 기반으로 작성되어야 하며, 허위 정보는 절대 포함하지 마세요.
    - 템플릿 순서를 반드시 따를 필요는 없습니다. 
    - RFP의 내용에 따라 순서를 자유롭게 조정해도 되며, 하나의 템플릿을 여러 번 사용해도 무방합니다. risk_management_plan 다음 risk_management_plan
    
    반드시 준수할 사항:
    - 응답은 반드시 아래 구조에 맞는 JSON 형식으로만 작성해야 합니다.
    - 설명, 코드블록, 마크다운, 주석, 예시 등 JSON 이외의 모든 텍스트는 절대 포함하지 마세요.
    - JSON 형식 오류(쉼표 누락, 따옴표 오류 등)가 없도록 주의하세요.
    - JSON만 반환하세요. 다른 내용은 한 글자도 포함하지 마세요.
    [슬라이드 템플릿 구조]
    {json.dumps(slide_templates, ensure_ascii=False, indent=2)}
    """.strip()

    # user prompt
    user_prompt = f"""
    [RFP 원문]
    {rfp_text}
    
    [고객 방향성/강조]
    {user_direction or '없음'}
    
    (고객명: {client_name} / 제안서 제목: {proposal_title})
    [강조 키워드]: {', '.join(keywords) if keywords else '없음'}
    
    위 정보를 바탕으로 PPT 슬라이드별 'description', 'elements', 'needs_research'를 작성해주세요.
    """.strip()

    # GPT 호출
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=temperature
    )

    result_text = response.choices[0].message.content.strip()

    # 코드블럭 제거
    result_text = re.sub(r"^```(?:json)?\s*|\s*```$", "", result_text, flags=re.DOTALL)

    try:
        parsed = json.loads(result_text)
        return parsed
    except json.JSONDecodeError as e:
        raise ValueError(f"응답이 유효한 JSON 형식이 아닙니다:\n\n{result_text}\n\n에러: {e}")


In [131]:
# analyze_rfp test
slides_dict = analyze_rfp(user_inputs=user_inputs, slide_templates=PROPOSAL_SLIDE_TEMPLATES)
slides_dict

{'cover_page': {'elements': {}},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '본 제안서는 AI 모델 학습 및 평가 자동화 프로세스와 워크플로우 설계를 위한 프로젝트로, 고객의 요구사항과 시장 트렌드를 반영한 맞춤형 솔루션을 제시합니다. 슬라이드는 프로젝트 이해, 고객 요구사항, 시장 분석, 솔루션 개요, 구현 계획, 기대 효과 등으로 구성되어 있습니다.',
  'elements': {}},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': '본 제안서는 AI 모델의 학습 및 평가를 자동화하여 연구 효율성과 생산성을 향상시키고, 실험 재현성을 확보하기 위한 표준화된 워크플로우를 설계하는 것을 목표로 합니다. 이를 통해 데이터 관리 체계와 자동화 파이프라인을 구축하고, 연구자 중심의 유연하고 접근성 높은 실험 환경을 조성하여 AI 개발 및 활용을 확대할 수 있는 기반을 마련합니다.',
  'elements': {'핵심 목표': ['데이터 관리 체계 정립', '자동화 파이프라인 구축', '사용자 중심 실험 환경 조성'],
   '기대 효과': ['연구 효율성 및 생산성 향상', '실험 재현성 및 성능 비교 분석 강화', 'AI 개발 및 활용 확대']},
  'needs_research': ['현재 시장에서 AI 모델 학습 및 평가 자동화 솔루션의 주요 트렌드는 무엇인가?',
   '경쟁사들이 제공하는 유사 솔루션의 특징과 차별화 요소는 무엇인가?']},
 'project_understanding': {'slide_description': '프로젝트의 필요성과 배경을 설명하며, 고객의 상황과 과제를 명확히 인식하고 있

In [78]:
slides_dict

{'cover_page': {'elements': {'Title': 'AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025. 05. 15',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 로고'}},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['1. 제안 개요',
    '2. 프로젝트 이해',
    '3. 고객 요구사항 요약',
    '4. 시장 분석 및 경쟁사 벤치마크',
    '5. 솔루션 개요',
    '6. 기술 사양',
    '7. 구현 계획',
    '8. 기대 효과',
    '9. 팀 소개',
    '10. 차별화 요소',
    '11. 결론 및 제안']}},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': '본 제안서는 AI 모델 학습 및 평가의 자동화를 통해 연구 효율성과 생산성을 극대화하고, 일관성 있는 개발 환경을 제공하기 위한 것입니다. EY 컨설팅은 고객사의 요구에 맞춘 맞춤형 설계를 통해 경쟁사 대비 우위를 확보할 수 있도록 지원합니다.',
  'elements': {'Title': '제안 개요',
   'MiddleText': 'AI 모델 학습 및 평가 자동화의 필요성과 EY 컨설팅의 솔루션',
   'SummaryPoints': ['AI 모델 학습 및 평가 자동화 프로세스 설계',
    '데이터 관리 및 학습 인프라 구축',
    '연구 효율성 및 생산성 향상',
    

In [109]:
import openai
import pandas as pd

# 슬라이드 딕셔너리 구조를 전개하여 테이블 형태로 변환
def flatten_slides_dict(slides_dict):
    slide_rows = []
    for sid, slide in slides_dict.items():
        if isinstance(slide, dict):
            slide_rows.append({
                "slide_id": sid,
                "description": slide.get("description", ""),
                "elements": slide.get("elements", {})
            })
    return pd.DataFrame(slide_rows)

def generate_research_questions_from_elements_v2(slide_title, description, elements_dict):
    element_summary = "\n".join([f"{k}: {v}" for k, v in elements_dict.items()])
    prompt = (
        "당신은 RFP 제안서를 위한 AI 기반 검증 질문 생성 도우미입니다.\n"
        "아래는 하나의 슬라이드 내용입니다. 이 내용 중 외부 리서치로 검증이 필요한 요소들을 식별하여 질문 형식으로 정리해 주세요.\n"
        "- 수치, 통계, 시장 분석, 산업 동향, 벤치마킹 등의 항목을 중심으로 질문을 만들어야 합니다.\n"
        "- 질문이 필요 없는 페이지의 경우 제외해주세요.\n"
        "- 확인이 필요한 사항을 명확히 작성해주세요 ex). 기업A -> 애플, 경쟁사: 제안서 상의 관련있는 기업.\n"
        "- 질문은 한 슬라이드당 1~3개로 제한합니다.\n\n"
        f"[슬라이드 제목]: {slide_title}\n"
        f"[슬라이드 설명]: {description}\n"
        f"[슬라이드 요소]:\n{element_summary}\n\n"
        "이 슬라이드 내용을 검증하기 위한 외부 리서치 질문은?"
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    result = response.choices[0].message.content.strip()
    return [line.strip("-•1234567890. ").strip() for line in result.split("\n") if "?" in line]

def generate_needs_research_table_from_df(slides_df):
    needs_list = []
    for _, row in slides_df.iterrows():
        title = row.get("elements", {}).get("Title", "")
        desc = row.get("description", "")
        elements = row.get("elements", {})
        questions = generate_research_questions_from_elements_v2(title, desc, elements)
        needs_list.append(questions)
    slides_df["needs_research"] = needs_list
    return slides_df


In [115]:
# 2. 필요 시 research 질문 자동 생성
slides_df = generate_needs_research_table_from_df(slides_df)

In [116]:
# 3. 출력 함수
def display_slides_with_research(slides_df):
    for idx, row in slides_df.iterrows():
        print(f"\n📌 Slide ID: {row['slide_id']}")
        print(f"설명: {row.get('description', '[없음]')}")
        print("🔍 리서치 질문:")
        if isinstance(row.get("needs_research", []), list) and row["needs_research"]:
            for q in row["needs_research"]:
                print(f" - {q}")
        else:
            print(" - (없음)")
display_slides_with_research(slides_df)


📌 Slide ID: cover_page
설명: 
🔍 리서치 질문:
 - "AI 모델 학습·평가 자동화 프로세스"에 대한 최신 산업 동향은 무엇인가요? 현재 시장에서 일반적으로 사용되는 자동화 프로세스의 사례를 제시해 주세요
 - EY 컨설팅이 AI 모델 학습 및 평가 자동화 분야에서 제공하는 서비스가 경쟁사와 비교했을 때 어떤 차별점을 가지고 있는지에 대한 벤치마킹 자료가 있나요?

📌 Slide ID: table_of_contents
설명: 
🔍 리서치 질문:
 - (없음)

📌 Slide ID: executive_summary
설명: 본 제안서는 AI 모델 학습 및 평가의 자동화를 통해 연구 효율성과 생산성을 극대화하고, 일관성 있는 개발 환경을 제공하기 위한 것입니다. EY 컨설팅은 고객사의 요구에 맞춘 맞춤형 설계를 통해 경쟁사 대비 우위를 확보할 수 있도록 지원합니다.
🔍 리서치 질문:
 - AI 모델 학습 및 평가 자동화가 연구 효율성과 생산성을 얼마나 향상시키는지에 대한 최신 통계나 연구 결과가 있는가?
 - EY 컨설팅이 AI 모델 학습 및 평가 자동화 분야에서 경쟁사 대비 어떤 우위를 가지고 있는지에 대한 시장 분석 자료가 있는가?
 - AI 모델의 실질적 성능 향상 및 산업 현장 적용을 가속화하는 데 있어 EY 컨설팅의 맞춤형 설계가 실제로 효과적이라는 것을 뒷받침할 수 있는 사례 연구나 벤치마킹 자료가 있는가?

📌 Slide ID: project_understanding
설명: AI 기술의 수요 증가는 데이터 관리 및 학습 인프라의 체계적 구축을 요구하고 있습니다. 다양한 도메인에서 생성되는 대규모 비정형 데이터를 신속하게 정제하고, 학습 가능한 구조로 가공할 수 있는 체계가 필요합니다. 반복적이고 복잡한 AI 모델 학습·평가·배포 과정을 자동화함으로써 연구 효율성과 생산성을 향상시키는 것이 목표입니다.
🔍 리서치 질문:
 - 현재 AI 기술의 수요 증가는 어느 정도이며, 이로 인해 데이터 관리 및 학습 인프라 구축의 필요성이 증가하고

In [114]:
# 1. slides_dict → DataFrame으로 변환 (description 포함)
slides_df = flatten_slides_dict(slides_dict)

In [117]:
def inject_research_questions_to_dict(slides_dict, slides_df):
    """
    slides_df의 needs_research 값을 slide_id 기준으로 slides_dict에 다시 삽입
    """
    for _, row in slides_df.iterrows():
        sid = row["slide_id"]
        questions = row.get("needs_research", [])
        if isinstance(questions, str):
            # 문자열로 되어 있는 경우 리스트로 변환 (ex. "['질문1', '질문2']")
            import ast
            try:
                questions = ast.literal_eval(questions)
            except Exception:
                questions = [questions]
        if isinstance(questions, list):
            slides_dict[sid]["needs_research"] = questions
    return slides_dict


In [118]:
slides_dict_updated = inject_research_questions_to_dict(slides_dict, slides_df)

In [119]:
slides_dict_updated

{'cover_page': {'elements': {'Title': 'AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025. 05. 15',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 로고'},
  'needs_research': ['"AI 모델 학습·평가 자동화 프로세스"에 대한 최신 산업 동향은 무엇인가요? 현재 시장에서 일반적으로 사용되는 자동화 프로세스의 사례를 제시해 주세요',
   'EY 컨설팅이 AI 모델 학습 및 평가 자동화 분야에서 제공하는 서비스가 경쟁사와 비교했을 때 어떤 차별점을 가지고 있는지에 대한 벤치마킹 자료가 있나요?']},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['1. 제안 개요',
    '2. 프로젝트 이해',
    '3. 고객 요구사항 요약',
    '4. 시장 분석 및 경쟁사 벤치마크',
    '5. 솔루션 개요',
    '6. 기술 사양',
    '7. 구현 계획',
    '8. 기대 효과',
    '9. 팀 소개',
    '10. 차별화 요소',
    '11. 결론 및 제안']},
  'needs_research': []},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': '본 제안서는 AI 모델 학습 및 평가의 자동화를 통해 연구 효율성과 생산성을 극대화하고, 일관성 있는 개발 환경을 제공하기 위한 것입니다. EY 컨설팅은 고객사의 요구에 맞춘 맞춤형 설계

In [120]:
def search_perplexity(query, api_key):
    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "sonar-pro",
        "messages": [{"role": "user", "content": query}],
        "return_citations": True  # 중요: citation 포함하도록 설정
    }

    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        try:
            data = response.json()
            message = data.get("choices", [{}])[0].get("message", {})
            content = message.get("content", "")

            # ✅ 실제 citation URL은 'search_results' 필드에 있음
            search_results = data.get("search_results", [])
            urls = []
            for result in search_results:
                url = result.get("url")
                if url:
                    urls.append(url)

            return content.strip(), urls
        except Exception as e:
            print("Perplexity Parsing Error:", e)
            return "", []
    else:
        print("Perplexity API Error:", response.status_code, response.text)
        return "", []




def search_serpapi_with_url(query, serpapi_key):
    """
    SerpAPI를 사용하여 Google 검색 결과 요약과 함께 URL 반환
    """
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": serpapi_key
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        results = response.json()
        snippets = []
        urls = []
        for r in results.get("organic_results", [])[:3]:
            snippet = r.get("snippet", "")
            link = r.get("link", "")
            if snippet and link:
                snippets.append(snippet)
                urls.append(link)
        return "\n".join(snippets), urls
    else:
        print("SerpAPI Error:", response.status_code, response.text)
        return "", []

def extract_relevant_summary_from_content(slide_description, elements, research_result):
    """
    리서치 결과 중 slide_description 및 elements 내용에 부합하는 핵심 정보만 요약 추출
    단, 요약은 지나치게 축약되지 않도록 3~5문장 내외로 간결히 작성
    """
    prompt = (
        f"당신은 제안서 작성 지원 AI입니다.\n"
        f"다음은 슬라이드의 목적과 포함된 요소들, 그리고 리서치 결과입니다.\n\n"
        f"[슬라이드 목적]\n{slide_description}\n\n"
        f"[슬라이드 요소]\n{elements}\n\n"
        f"[리서치 결과]\n{research_result}\n\n"
        f"위 슬라이드 목적과 요소에 부합하는 핵심 정보만 간결히 요약하세요. 불필요한 일반론, 도입부, 중복 표현은 제거하세요. 요약은 3~5문장으로 유지하세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


def check_relevance(description, research_result):
    """
    리서치 결과가 slide_description과 논리적으로 부합하는지 확인
    """
    prompt = (
        f"당신은 제안서 전략 검토 전문가입니다.\n"
        f"다음은 슬라이드의 목적(description)과 검색을 통해 얻은 리서치 결과입니다.\n"
        f"[슬라이드 목적]\n{description}\n\n"
        f"[리서치 결과]\n{research_result}\n\n"
        f"이 리서치 결과가 위 슬라이드 목적을 충분히 뒷받침합니까? "
        f"'네' 또는 '아니오'로 시작하고, 반드시 이유를 1문장 이상 포함하세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1
    )
    answer = response.choices[0].message.content.strip()
    print(f"\n[GPT 판단 결과]\n{answer}\n")
    return answer.startswith("네")

def regenerate_query(original_query, previous_answer):
    """
    검색 결과가 부적합한 경우, 질문을 더 명확하게 만들어주는 로직
    """
    prompt = (
        f"다음은 정보 검색을 위한 원래 질문과 검색 결과입니다.\n\n"
        f"[질문]\n{original_query}\n\n"
        f"[검색 결과]\n{previous_answer}\n\n"
        "위 결과가 부정확하거나 부족한 경우, 질문을 더 구체적이고 답변을 유도할 수 있도록 재작성하세요. "
        "새 질문은 1문장으로 간결하게 작성해 주세요."
    )
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

#     return completed_results
def smart_research_fill(slides_dict, search_mode="perplexity", max_retry=2):
    """
    슬라이드에 대해 스마트 리서치를 수행하고, 불충분한 경우 재질문하여 더 나은 결과를 시도함.
    Perplexity 사용 시 citation URL도 함께 저장됨.
    """
    completed_results = {}

    for slide_id, slide in slides_dict.items():
        desc = slide.get("description", "")
        if not desc:
            print(f"[경고] description이 비어 있습니다: slide_id={slide_id}")
            continue

        needs_list = slide.get("needs_research", [])
        if not needs_list:
            continue

        slide_results = {}
        for need in needs_list:
            query = need
            attempt = 0

            while attempt < max_retry:
                # 검색 수행
                if search_mode == "perplexity":
                    content, urls = search_perplexity(query, perplexity_api_key)
                elif search_mode == "serpapi":
                    content, urls = search_serpapi_with_url(query, serp_api_key)
                else:
                    raise ValueError("search_mode는 'perplexity' 또는 'serpapi'만 지원됩니다.")

                # 실패 시 중단
                if not content:
                    print(f"[실패] 검색 결과 없음 - query: {query}")
                    break

                # 관련성 판단 통과 후 요약 삽입
                if check_relevance(desc, content):
                    summary = extract_relevant_summary_from_content(
                        slide_description=desc,
                        elements=slide.get("elements", {}),
                        research_result=content
                    )
                    slide_results[need] = {
                        "content": summary,
                        "urls": urls
                    }
                    break  # 성공 → 다음 need로

                else:
                    query = regenerate_query(query, content)
                    attempt += 1
                    time.sleep(1)

        if slide_results:
            completed_results[slide_id] = {
                "slide_title": slide.get("slide_title", ""),
                "research_results": slide_results
            }

    return completed_results



def merge_research_results_into_slides(slides_dict, research_results_dict):
    """
    slides_dict 구조에 research_results_dict 데이터를 병합
    slides_dict[slide_id]['research_results'] 로 삽입
    """
    for slide_id, data in research_results_dict.items():
        if slide_id in slides_dict:
            slides_dict[slide_id]['research_results'] = data.get('research_results', {})
    return slides_dict


In [121]:
research_results_dict = smart_research_fill(slides_dict_updated, search_mode="perplexity")

slides_dict_updated = merge_research_results_into_slides(
    slides_dict=slides_dict_updated,
    research_results_dict=research_results_dict
)

slides_dict_updated

[경고] description이 비어 있습니다: slide_id=cover_page
[경고] description이 비어 있습니다: slide_id=table_of_contents

[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. AI 모델 학습 및 평가의 자동화가 연구 효율성과 생산성을 크게 향상시킬 수 있다는 점과, 이를 통해 경쟁사 대비 우위를 확보할 수 있다는 점이 명확히 제시되어 있습니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. EY 컨설팅의 독자적 AI 플랫폼과 맞춤형 전략, 실용적 접근법, 그리고 글로벌 네트워크를 통한 신속한 적용은 AI 모델 학습 및 평가 자동화를 통해 연구 효율성과 생산성을 극대화하고 일관성 있는 개발 환경을 제공하려는 목적에 부합합니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. EY 컨설팅의 맞춤형 AI 설계와 컨설팅이 실제 산업 현장에서 성능과 효율성 향상에 기여한 사례와 벤치마킹 도구를 통해 연구 효율성과 생산성을 극대화하고 일관성 있는 개발 환경을 제공하는 데 효과적임을 입증하고 있기 때문입니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. AI 기술 수요 증가와 이에 따른 데이터 관리 및 학습 인프라의 중요성이 강조되었으며, 이는 슬라이드에서 제시한 체계적 구축의 필요성과 연구 효율성 향상의 목표와 일치합니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. 다양한 도메인에서 비정형 데이터를 신속하게 정제하고 가공할 수 있는 체계의 필요성과 이를 구현한 실제 벤치마킹 사례를 제시함으로써, AI 기술의 수요 증가에 따른 데이터 관리 및 학습 인프라 구축의 중요성을 강조하고 있습니다.


[GPT 판단 결과]
네, 이 리서치 결과는 슬라이드 목적을 충분히 뒷받침합니다. AI 모델의 자동화가 연구 효율성과 생산성 향상에 기

{'cover_page': {'elements': {'Title': 'AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025. 05. 15',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 로고'},
  'needs_research': ['"AI 모델 학습·평가 자동화 프로세스"에 대한 최신 산업 동향은 무엇인가요? 현재 시장에서 일반적으로 사용되는 자동화 프로세스의 사례를 제시해 주세요',
   'EY 컨설팅이 AI 모델 학습 및 평가 자동화 분야에서 제공하는 서비스가 경쟁사와 비교했을 때 어떤 차별점을 가지고 있는지에 대한 벤치마킹 자료가 있나요?']},
 'table_of_contents': {'slide_description': '전체 제안서의 슬라이드 구성을 한눈에 파악할 수 있도록 시각적으로 정리합니다.',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['1. 제안 개요',
    '2. 프로젝트 이해',
    '3. 고객 요구사항 요약',
    '4. 시장 분석 및 경쟁사 벤치마크',
    '5. 솔루션 개요',
    '6. 기술 사양',
    '7. 구현 계획',
    '8. 기대 효과',
    '9. 팀 소개',
    '10. 차별화 요소',
    '11. 결론 및 제안']},
  'needs_research': []},
 'executive_summary': {'slide_description': '제안서 전체의 핵심 내용을 1~2페이지 내에 요약하여 임원 또는 의사결정자가 빠르게 이해할 수 있도록 구성합니다.',
  'description': '본 제안서는 AI 모델 학습 및 평가의 자동화를 통해 연구 효율성과 생산성을 극대화하고, 일관성 있는 개발 환경을 제공하기 위한 것입니다. EY 컨설팅은 고객사의 요구에 맞춘 맞춤형 설계

In [125]:
def refine_slides_dict(slides_dict, rfp_text, user_inputs):
    refined_slides = deepcopy(slides_dict)

    for slide_id, slide in slides_dict.items():
        try:
            prompt = f"""
            당신은 맥킨지 수준의 컨설팅 제안서 전략 전문가입니다.
            
            당신의 역할은 제안서 슬라이드의 내용을 전략적으로 더 설득력 있고 고도화된 형태로 다듬는 것입니다.
            
            --- 컨텍스트 ---
            클라이언트: {user_inputs['client_name']}
            제안서 제목: {user_inputs['proposal_title']}
            문체 및 톤: {user_inputs['style']}
            작성 방향: {user_inputs['user_direction']}
            강조할 키워드: {", ".join(user_inputs['keywords'])}
            RFP 원문: {rfp_text}
            
            --- 슬라이드 정보 ---
            슬라이드 ID: {slide_id}
            슬라이드 제목: {slide.get("slide_title", "")}
            현재 슬라이드 설명: {slide.get("description", "")}
            슬라이드 요소: {slide.get("elements", {})}
            리서치 결과: {slide.get("research_results", {})}
            
            --- 작업 지시 ---
            다음의 정확한 키를 포함하는 유효한 Python 딕셔너리 하나만 반환하세요:
            - "description": 현재 슬라이드 설명을 기반으로 더욱 전략적이고 설득력 있게 다듬은 설명 (3~5문장)
            - "elements": 기존 요소를 기반으로 논리적 구조와 설득력을 강화한 내용. 특히 리스트, 표, 그래프 등은 Research Results를 기반으로 실제 수치를 반영하여 구체적으로 작성하세요.
            
            [중요] 표나 그래프가 쓰이는 경우 아래와 같이 데이터를 수정해주세요:
            - 표는 리스트로 구성 및 수정하세요. 예: [["헤더1", "헤더2"], ["값1", "값2"], ...]
            - 그래프는 다음 형식으로 작성하세요. 반드시 적절한 그래프 타입을 데이터 구조에 맞게 지정해야 합니다, 기존에 있던 데이터가 맞지 않으면 과감하게 수정해주세요:
              "GraphLeft": {{
                "description": "그래프 설명",
                "graph_type": "Line Chart" 또는 "Bar Chart" 또는 "Pie Chart" 중 하나,
                "data_source": "출처 예: 통계청, 2024",
                "data_table": [["헤더1", "헤더2"], ["값1", "값2"], ...]
              }}
            
            반드시 리서치 결과를 바탕으로 수치를 작성하고, 자료 출처를 명시해야 합니다.
            그래프나 표가 필요 없는 경우 작성하지 마세요. ex). cover_page에 data_table 혹은 graph가 들어가면 안됨
            
            다음 조건을 꼭 지켜주세요:
            - 마크다운, 인용 부호, 코드 블록, 주석 등을 추가하지 마세요
            - 반드시 한글로만 작성하세요
            - 반환 형식은 Python 딕셔너리 하나만 출력하세요
            """

            response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt.strip()}],
                temperature=0.3
            )
            result_text = response.choices[0].message.content.strip()
            
            # 안전한 파싱 (JSON이 아닌 파이썬 dict로)
            parsed = ast.literal_eval(result_text)

            refined_slides[slide_id]["slide_description"] = parsed.get("slide_description", "nan")
            refined_slides[slide_id]["elements"] = parsed.get("elements", {})

        except Exception as e:
            print(f"[고도화 실패] {slide_id}: {e}")
            refined_slides[slide_id]["slide_description"] = "nan"
            refined_slides[slide_id]["elements"] = {}

    return refined_slides


In [126]:
refined = refine_slides_dict(
    slides_dict=slides_dict_updated,
    rfp_text=rfp_text,
    user_inputs=user_inputs,
)

In [127]:
refined

{'cover_page': {'elements': {'Title': 'AI 모델 학습·평가 자동화 프로세스 및 워크플로우 설계',
   'Subtitle': 'EY 컨설팅',
   'ProjectDate': '2025. 05. 15',
   'PreparedBy': 'EY 컨설팅',
   'Logo': 'EY 로고'},
  'needs_research': ['"AI 모델 학습·평가 자동화 프로세스"에 대한 최신 산업 동향은 무엇인가요? 현재 시장에서 일반적으로 사용되는 자동화 프로세스의 사례를 제시해 주세요',
   'EY 컨설팅이 AI 모델 학습 및 평가 자동화 분야에서 제공하는 서비스가 경쟁사와 비교했을 때 어떤 차별점을 가지고 있는지에 대한 벤치마킹 자료가 있나요?'],
  'slide_description': 'nan'},
 'table_of_contents': {'slide_description': 'nan',
  'description': '',
  'elements': {'Title': '목차',
   'SectionList': ['1. 제안 개요',
    '2. 프로젝트 이해',
    '3. 고객 요구사항 요약',
    '4. 시장 분석 및 경쟁사 벤치마크',
    '5. 솔루션 개요',
    '6. 기술 사양',
    '7. 구현 계획',
    '8. 기대 효과',
    '9. 팀 소개',
    '10. 차별화 요소',
    '11. 결론 및 제안']},
  'needs_research': []},
 'executive_summary': {'slide_description': 'nan',
  'description': '본 제안서는 AI 모델 학습 및 평가의 자동화를 통해 연구 효율성과 생산성을 극대화하고, 일관성 있는 개발 환경을 제공하기 위한 것입니다. EY 컨설팅은 고객사의 요구에 맞춘 맞춤형 설계를 통해 경쟁사 대비 우위를 확보할 수 있도록 지원합니다.',
  'elements': {'Title': '제안 개요',
  

In [ ]:
import json
from datetime import datetime
# 오늘 날짜를 포함한 파일명 생성
today_str = datetime.now().strftime("%Y%m%d_%H%M")
file_name = f"proposal_{today_str}.json"

# JSON 파일로 저장
with open(file_name, "w", encoding="utf-8") as f:
    json.dump(slides_dict, f, indent=2, ensure_ascii=False)

print(f"✅ JSON 저장 완료: {file_name}")


In [88]:
# slides = refined
# for slide_id, slide_info in slides_dict.items():
#    print(slide_id, slide_info)
# # slides는 refined 상태라고 가정
# ordered_keys = [
#     "cover_page",
#     "table_of_contents",
#     "project_understanding",
#     "client_needs_summary",
#     "market_analysis_market_overview",
#     "growth_trend_analysis",
#     "industry_drivers_challenges",
#     "competitive_benchmarking",
#     "swot_analysis",
#     "solution_overview",
#     "strategic_recommendations",
#     "implementation_plan",
#     "timeline_milestones",
#     "expected_benefits",
#     "investment_budget_estimation",
#     "risk_management_plan",
#     "team_introduction",
#     "why_us_differentiation",
#     "qna",
#     "closing_summary"
# ]

# # slides를 순서에 맞게 재정렬
# slides = refined  # 기존 슬라이드 딕셔너리
# sorted_slides = {k: slides[k] for k in ordered_keys if k in slides}

# # 결과 확인
# for slide_id, slide_info in sorted_slides.items():
#     print(slide_id, slide_info)


{'cover_page': {'slide_title': '커버 페이지',
  'elements': {'Title': '메리츠종금증권 웹접근성 개선 프로젝트',
   'Subtitle': 'EY 컨설팅의 맞춤형 시스템 제안',
   'Key Highlights': ['타사 비교를 통한 벤치마크 분석',
    '자체적인 접근성 개선 전략',
    '현실적이고 실행 가능한 설계'],
   'Confidentiality Notice': '본 제안서는 메리츠종금증권 웹접근성 개선 프로젝트를 위한 목적으로만 사용되며, 외부 또는 타사에 배포할 수 없습니다.'},
  'slide_description': "This proposal outlines a strategic approach to enhancing web accessibility for Meritz Securities, leveraging EY Consulting's expertise. Our solution is grounded in comprehensive benchmarking and comparative analysis with industry peers, ensuring a robust and sustainable implementation. We aim to deliver a tailored system that aligns with Meritz Securities' operational realities and strategic objectives."},
 'table_of_contents': {'slide_title': '목차',
  'elements': {'I. 사업개요': {'1. 추진배경 및 목표': '프로젝트의 필요성과 목표를 명확히 정의하여 이해 관계자 모두가 공감할 수 있는 방향성을 제시합니다.',
    '2. 사업내용': '프로젝트의 주요 활동과 기대 효과를 구체적으로 설명하여 실현 가능한 결과를 도출합니다.'},
   'II. 제안요청 내용': {'1. 제안 기본사항': '제안의 